In [58]:
# ========================================================
# Author: David Mihola (david.mihola@student.tugraz.at)
# Matrikelnummer: 12211951
# Date: 21. 12. 2022
# ========================================================

In [1]:
import tensorflow.keras.datasets as tfd
import tensorflow.keras.utils as tfu
import tensorflow.keras.models as tfm
import tensorflow.keras.layers as tfl
import tensorflow.keras.callbacks as tfc
import tensorflow as tf
import numpy as np

2022-12-06 12:56:57.657234: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-06 12:56:57.844858: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-06 12:56:57.921302: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-06 12:56:58.630814: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [2]:
(x_train, y_train), (x_test, y_test) = tfd.cifar10.load_data()
x_train, x_test = np.mean(x_train, axis=3), np.mean(x_test, axis=3) # convert to grayscale
x_train, x_test = x_train / 255, x_test / 255 # normalize to pixel values between 0 and 1
x_train, x_test = tf.expand_dims(x_train, -1), tf.expand_dims(x_test, -1) # adding chanel dimension

NUM_OF_CLASSES = 10
y_train = tfu.to_categorical(y_train, num_classes=NUM_OF_CLASSES)
y_test = np.array(y_test).reshape(len(y_test))
x_train.shape, y_train.shape, x_test.shape, y_test.shape

170498071/170498071 [==============================] - 47s 0us/step


2022-12-06 12:57:55.286370: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-06 12:57:55.355317: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-12-06 12:57:55.355333: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-12-06 12:57:55.356544: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN

(TensorShape([50000, 32, 32, 1]),
 (50000, 10),
 TensorShape([10000, 32, 32, 1]),
 (10000,))

In [61]:
full_conv = tfm.Sequential([
    tfl.Conv2D(filters=16, kernel_size=(3, 3), activation="relu", padding="same"),
    tfl.Conv2D(filters=16, kernel_size=(3, 3), strides=(2, 2), activation="relu", padding="same"),
    tfl.Conv2D(filters=32, kernel_size=(3, 3), activation="relu", padding="same"),
    tfl.Conv2D(filters=32, kernel_size=(3, 3), strides=(2, 2), activation="relu", padding="same"),
    tfl.Conv2D(filters=64, kernel_size=(3, 3), activation="relu", padding="same"),
    tfl.Conv2D(filters=64, kernel_size=(3, 3), strides=(2, 2), activation="relu", padding="same"),
    tfl.Conv2D(filters=128, kernel_size=(3, 3), activation="relu", padding="same"),
    tfl.Conv2D(filters=128, kernel_size=(3, 3), strides=(2, 2), activation="relu", padding="same"),
    tfl.Conv2D(filters=256, kernel_size=(3, 3), activation="relu", padding="same"),
    tfl.Conv2D(filters=256, kernel_size=(3, 3), strides=(2, 2), activation="relu", padding="same"),
    tfl.Conv2D(filters=10, kernel_size=(1, 1), activation="softmax", padding="same"),
    tfl.Reshape([10])                                                                               
])

model_full_conv_max_pool = tfm.Sequential([
    tfl.Conv2D(filters=32, kernel_size=(3, 3), activation="relu", padding="same"),
    tfl.MaxPool2D(),
    tfl.Conv2D(filters=64, kernel_size=(3, 3), activation="relu", padding="same"),
    tfl.MaxPool2D(),
    tfl.Conv2D(filters=128, kernel_size=(3, 3), activation="relu", padding="same"),
    tfl.MaxPool2D(),
    tfl.Conv2D(filters=256, kernel_size=(3, 3), activation="relu", padding="same"),
    tfl.MaxPool2D(),
    tfl.Conv2D(filters=512, kernel_size=(3, 3), activation="relu", padding="same"),
    tfl.MaxPool2D(),
    tfl.Conv2D(filters=256, kernel_size=(1, 1), activation="relu", padding="same"),
    tfl.Conv2D(filters=10, kernel_size=(1, 1), activation="softmax", padding="same"),
    tfl.Reshape([10])
])

VGG_inspired_less_kernels = tfm.Sequential([
    tfl.Conv2D(filters=16, kernel_size=(3, 3), activation="relu", padding="same"),
    tfl.Conv2D(filters=16, kernel_size=(3, 3), activation="relu", padding="same"),
    tfl.MaxPool2D(),
    tfl.Conv2D(filters=32, kernel_size=(3, 3), activation="relu", padding="same"),
    tfl.Conv2D(filters=32, kernel_size=(3, 3), activation="relu", padding="same"),
    tfl.MaxPool2D(),
    tfl.Conv2D(filters=64, kernel_size=(3, 3), activation="relu", padding="same"),
    tfl.Conv2D(filters=64, kernel_size=(3, 3), activation="relu", padding="same"),
    tfl.MaxPool2D(),
    tfl.Flatten(),
    tfl.Dense(128, activation="relu"),
    tfl.Dense(10, activation="softmax")
])

VGG_inspired_more_kernels = tfm.Sequential([
    tfl.Conv2D(filters=32, kernel_size=(3, 3), activation="relu", padding="same"),
    tfl.Conv2D(filters=32, kernel_size=(3, 3), activation="relu", padding="same"),
    tfl.MaxPool2D(),
    tfl.Conv2D(filters=64, kernel_size=(3, 3), activation="relu", padding="same"),
    tfl.Conv2D(filters=64, kernel_size=(3, 3), activation="relu", padding="same"),
    tfl.MaxPool2D(),
    tfl.Conv2D(filters=128, kernel_size=(3, 3), activation="relu", padding="same"),
    tfl.Conv2D(filters=128, kernel_size=(3, 3), activation="relu", padding="same"),
    tfl.MaxPool2D(),
    tfl.Flatten(),
    tfl.Dense(256, activation="relu"),
    tfl.Dense(10, activation="softmax")
])

VGG_inspired_3_dense_layers = tfm.Sequential([
    tfl.Conv2D(filters=32, kernel_size=(3, 3), activation="relu", padding="same"),
    tfl.Conv2D(filters=32, kernel_size=(3, 3), activation="relu", padding="same"),
    tfl.MaxPool2D(),
    tfl.Conv2D(filters=64, kernel_size=(3, 3), activation="relu", padding="same"),
    tfl.Conv2D(filters=64, kernel_size=(3, 3), activation="relu", padding="same"),
    tfl.MaxPool2D(),
    tfl.Conv2D(filters=128, kernel_size=(3, 3), activation="relu", padding="same"),
    tfl.Conv2D(filters=128, kernel_size=(3, 3), activation="relu", padding="same"),
    tfl.Flatten(),
    tfl.Dense(512, activation="relu"),
    tfl.Dense(128, activation="relu"),
    tfl.Dense(10, activation="softmax")
])

full_conv.compile(optimizer="adam", loss="categorical_crossentropy")
model_full_conv_max_pool.compile(optimizer="adam", loss="categorical_crossentropy")
VGG_inspired_less_kernels.compile(optimizer="adam", loss="categorical_crossentropy")
VGG_inspired_more_kernels.compile(optimizer="adam", loss="categorical_crossentropy")
VGG_inspired_3_dense_layers.compile(optimizer="adam", loss="categorical_crossentropy")

In [62]:
full_conv.fit(x_train, y_train, epochs=100, validation_split=0.2, batch_size=32,
              callbacks=[tfc.EarlyStopping(monitor="val_loss", patience=3, mode="min", restore_best_weights=True)])
model_full_conv_max_pool.fit(x_train, y_train, epochs=100, validation_split=0.2, batch_size=32,
                             callbacks=[tfc.EarlyStopping(monitor="val_loss", patience=3, mode="min", restore_best_weights=True)])
VGG_inspired_less_kernels.fit(x_train, y_train, epochs=100, validation_split=0.2, batch_size=32,
                          callbacks=[tfc.EarlyStopping(monitor="val_loss", patience=3, mode="min", restore_best_weights=True)])
VGG_inspired_more_kernels.fit(x_train, y_train, epochs=100, validation_split=0.2, batch_size=32,
                          callbacks=[tfc.EarlyStopping(monitor="val_loss", patience=3, mode="min", restore_best_weights=True)])
VGG_inspired_3_dense_layers.fit(x_train, y_train, epochs=100, validation_split=0.2, batch_size=32,
                                callbacks=[tfc.EarlyStopping(monitor="val_loss", patience=3, mode="min", restore_best_weights=True)])

1250/1250 [==============================] - 254s 202ms/step - loss: 1.7157 - val_loss: 1.3809


In [63]:
full_conv.save_weights("./models/full_conv")
model_full_conv_max_pool.save_weights("./models/model_full_conv_max_pool")
VGG_inspired_less_kernels.save_weights("./models/VGG_inspired_less_kernels")
VGG_inspired_more_kernels.save_weights("./models/VGG_inspired_more_kernels")
VGG_inspired_3_dense_layers.save_weights("./models/VGG_inspired_3_dense_layers")

In [64]:
full_conv_pred = full_conv.predict(x_test)
model_full_conv_max_pool_pred = model_full_conv_max_pool.predict(x_test)
VGG_inspired_less_kernels_pred = VGG_inspired_less_kernels.predict(x_test)
VGG_inspired_more_kernels_pred = VGG_inspired_more_kernels.predict(x_test)
VGG_inspired_3_dense_layers_pred = VGG_inspired_3_dense_layers.predict(x_test)

313/313 [==============================] - 14s 44ms/step


In [65]:
NUM_OF_TEST_SAMPLES = len(y_test)

full_conv_accuracy = (np.argmax(full_conv_pred, axis=1) == y_test).sum() / NUM_OF_TEST_SAMPLES
model_full_conv_max_pool_accuracy = (np.argmax(model_full_conv_max_pool_pred, axis=1) == y_test).sum() / NUM_OF_TEST_SAMPLES
VGG_inspired_less_kernels_accuracy = (np.argmax(VGG_inspired_less_kernels_pred, axis=1) == y_test).sum() / NUM_OF_TEST_SAMPLES
VGG_inspired_more_kernels_accuracy = (np.argmax(VGG_inspired_more_kernels_pred, axis=1) == y_test).sum() / NUM_OF_TEST_SAMPLES
VGG_inspired_3_dense_layers_accuracy = (np.argmax(VGG_inspired_3_dense_layers_pred, axis=1) == y_test).sum() / NUM_OF_TEST_SAMPLES

print(f"full_conv_accuracy:                   {full_conv_accuracy * 100:.2f} %")
print(f"model_full_conv_max_pool_accuracy:    {model_full_conv_max_pool_accuracy * 100:.2f} %")
print(f"VGG_inspired_less_kernels_accuracy:       {VGG_inspired_less_kernels_accuracy * 100:.2f} %")
print(f"VGG_inspired_more_kernels_accuracy:       {VGG_inspired_more_kernels_accuracy * 100:.2f} %")
print(f"VGG_inspired_3_dense_layers_accuracy: {VGG_inspired_3_dense_layers_accuracy * 100:.2f} %")

full_conv_accuracy:                   44.27 %
model_full_conv_max_pool_accuracy:    53.76 %
VGG_inspired_less_kernels_accuracy:       47.18 %
VGG_inspired_more_kernels_accuracy:       55.80 %
VGG_inspired_3_dense_layers_accuracy: 49.82 %
